# Donkey Colab with GPU

Donkey Colab with GPU.

In [1]:
!git clone -b dev https://github.com/autorope/donkeycar.git donkey

Cloning into 'donkey'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 6688 (delta 0), reused 0 (delta 0), pack-reused 6687
Receiving objects: 100% (6688/6688), 41.53 MiB | 34.60 MiB/s, done.
Resolving deltas: 100% (3853/3853), done.


In [2]:
!pip3 install -e donkey

Obtaining file:///content/donkey
    100% |████████████████████████████████| 51kB 2.4MB/s 
    100% |████████████████████████████████| 225kB 8.0MB/s 
    100% |████████████████████████████████| 122kB 33.0MB/s 
    100% |████████████████████████████████| 194kB 27.0MB/s 
  Running setup.py develop for donkeycar


In [3]:
!donkey createcar --template donkey2  /content/mycar

using donkey version: 2.5.7 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: donkey2
Copying car config defaults. Adjust these before starting your car.
Donkey setup complete.


In [5]:
!mkdir /content/mycar/data/tub001

!mv datas.zip /content/mycar/data/tub001/

mkdir: cannot create directory ‘/content/mycar/data/tub001’: File exists


In [6]:
%cd /content/mycar/data/tub001

!unzip /content/mycar/data/tub001/datas.zip

/content/mycar/data/tub001
Archive:  /content/mycar/data/tub001/datas.zip
  inflating: 1086_cam-image_array_.jpg  
  inflating: 1202_cam-image_array_.jpg  
  inflating: 955_cam-image_array_.jpg  
  inflating: 1075_cam-image_array_.jpg  
  inflating: record_997.json         
  inflating: 1468_cam-image_array_.jpg  
  inflating: record_978.json         
  inflating: 1666_cam-image_array_.jpg  
  inflating: 1695_cam-image_array_.jpg  
  inflating: 1411_cam-image_array_.jpg  
  inflating: record_1548.json        
  inflating: record_581.json         
  inflating: record_1118.json        
  inflating: 1288_cam-image_array_.jpg  
  inflating: 628_cam-image_array_.jpg  
  inflating: record_702.json         
  inflating: 1962_cam-image_array_.jpg  
  inflating: record_1231.json        
  inflating: record_352.json         
  inflating: 235_cam-image_array_.jpg  
  inflating: 1991_cam-image_array_.jpg  
  inflating: record_1661.json        
  inflating: record_1374.json        
  inflating: rec

In [8]:
#!/usr/bin/env python3

import os

from docopt import docopt
import donkeycar as dk
import tensorflow as tf
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras_tpu import KerasLinear
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.web_controller import LocalWebController
from donkeycar.parts.clock import Timestamp
from donkeycar.parts.datastore import TubGroup, TubWriter
#from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.transform import Lambda


def train(cfg, tub_names, new_model_path, base_model_path=None):
    """
    use the specified data in tub_names to train an artifical neural network
    saves the output trained model as model_name
    """
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    new_model_path = os.path.expanduser(new_model_path)

    kl = KerasLinear()
    if base_model_path is not None:
        base_model_path = os.path.expanduser(base_model_path)
        kl.load(base_model_path)

    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join("/content/mycar/data/", '*')
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys,
                                                    batch_size=128,
                                                    train_frac=0.8)

    total_records = len(tubgroup.df)
    total_train = int(total_records * 0.8)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train //128
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=new_model_path,
             steps=steps_per_epoch,
             train_split=0.8)


if __name__ == '__main__':
    #with tf.device('/gpu:0'):
      cfg = None

      tub = None
      new_model_path = "/content/mycar/models/mypilot"
      base_model_path = None
      cache = None
      train(cfg, tub, new_model_path, base_model_path)

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.22.19.42:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13195945044999046901)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14195895620642536384)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 10488754471525171438)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 2155608590753275289)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4043842383210292057)
INFO:tensorflow:*** Available Device: _Device

InvalidArgumentError: ignored

In [0]:
!mv /content/keras_tpu.py /content/donkey/donkeycar/parts/